# Testing possible atom mappings in perses
### Module to test how a hybrid system will be generated for a system

    * This does _not_ check if hydrogens are mapped or not. That is handled by class SmallMoleculeSetProposalEngine as a system needs to be generated and the hydrogens restraints need to be checked
    * AtomMapper also does not check that the minimal number of core atoms are retained by a map. If

In [12]:
# from perses.utils.openeye import createOEMolFromSDF
from perses.utils.smallmolecules import render_atom_mapping
from perses.utils.openeye import smiles_to_oemol
from perses.rjmc.topology_proposal import AtomMapper
from openeye import oechem
import itertools

In [57]:
##### molecules to play with

smiles = {'benzene':'c1ccccc1','phenol':'c1ccc(cc1)O','anisole':'c1ccc(cc1)OC','cyclohexane':'C1CCCCC1'}
smiles = {'benzene':'c1ccccc1','toluene':'c1ccccc1C','nitrobenzene':'C1=CC=C(C=C1)[N+](=O)[O-]','cyclohexane':'C1CCCCC1'}
list_of_mols = []
for name,smi in smiles.items():
    mol = smiles_to_oemol(smi, title=name)
    list_of_mols.append(mol)
    
print(f'{len(list_of_mols)} oemols have been generated')

4 oemols have been generated


#### Starting by just using the three default options in perses

In [75]:
# looping over default mapping strengths
names_and_oemols = dict(zip(smiles.keys(),list_of_mols))

print(names_and_oemols)
for map_strength in ['weak','default','strong']:
    print(f'Doing {map_strength} mapping')
    for nameA,nameB in itertools.combinations(names_and_oemols,2):
        molA = names_and_oemols[nameA]
        molB = names_and_oemols[nameB]
        Mapper = AtomMapper([molA,molB],map_strength=map_strength)
        if len(Mapper.atom_map) < 3:
            # too many unique atoms
            print(f'Cannot map {nameA} to {nameB} effectively with {map_strength} map strength')
        else:
            print(f'{nameA} ---> {nameB} '+u'\u2713')
            render_atom_mapping(f'{nameA}to{nameB}_{map_strength}.png', Mapper.current_molecule, Mapper.proposed_molecule, Mapper.atom_map)
    print()

{'benzene': <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0xa28904b40> >, 'toluene': <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0xa289049f0> >, 'nitrobenzene': <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0xa28904ed0> >, 'cyclohexane': <oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0xa289043f0> >}
Doing weak mapping
benzene ---> toluene ✓
benzene ---> nitrobenzene ✓
Cannot map benzene to cyclohexane effectively with weak map strength
toluene ---> nitrobenzene ✓
Cannot map toluene to cyclohexane effectively with weak map strength
Cannot map nitrobenzene to cyclohexane effectively with weak map strength

Doing default mapping
benzene ---> toluene ✓
benzene ---> nitrobenzene ✓
Cannot map benzene to cyclohexane effectively with default map strength
toluene ---> nitrobenzene ✓
Cannot map toluene to cyclohexane effectively with default map strength
Cannot map nitrobenzene to cyclohexane effectively with default ma

#### Now lets try a few different mapping schemes using openeye's `OEExprOpts`

https://docs.eyesopen.com/toolkits/python/oechemtk/OEChemConstants/OEExprOpts.html

In [86]:
# this just checks atoms are in rings of the same size, with no check for aromaticity
atom_expr = oechem.OEExprOpts_IntType
bond_expr = oechem.OEExprOpts_RingMember 


nameA = 'cyclohexane'

for nameB in names_and_oemols.keys():
    if nameB == nameA:
        continue
    molA = names_and_oemols[nameA]
    molB = names_and_oemols[nameB]    
    Mapper = AtomMapper([molA,molB],atom_expr=atom_expr,bond_expr=bond_expr)
    if len(Mapper.atom_map) < 3:
        # too many unique atoms
        print(f'Cannot map {nameA} to {nameB} effectively with {map_strength} map strength')
    else:
        print(f'{nameA} ---> {nameB} '+u'\u2713')
        render_atom_mapping(f'{nameA}to{nameB}.png', Mapper.current_molecule, Mapper.proposed_molecule, Mapper.atom_map)

INFO:proposal_generator:Using user defined atom_expr
INFO:proposal_generator:Any map_strength set has been ignored
INFO:proposal_generator:Using user defined bond_expr
INFO:proposal_generator:Any map_strength set has been ignored
INFO:proposal_generator:Using user defined atom_expr
INFO:proposal_generator:Any map_strength set has been ignored
INFO:proposal_generator:Using user defined bond_expr
INFO:proposal_generator:Any map_strength set has been ignored
INFO:proposal_generator:Using user defined atom_expr
INFO:proposal_generator:Any map_strength set has been ignored
INFO:proposal_generator:Using user defined bond_expr
INFO:proposal_generator:Any map_strength set has been ignored


cyclohexane ---> benzene ✓
cyclohexane ---> toluene ✓
cyclohexane ---> nitrobenzene ✓
